In [1]:
import torch

In [2]:
print(torch.cuda.is_available())

True


In [3]:
x = torch.Tensor(2,3,4)

In [4]:
x

tensor([[[-4.1344e-13,  4.5853e-41, -4.1344e-13,  4.5853e-41],
         [-4.1340e-13,  4.5853e-41, -4.1334e-13,  4.5853e-41],
         [-4.1345e-13,  4.5853e-41, -4.1343e-13,  4.5853e-41]],

        [[ 2.0179e-43,  0.0000e+00,  1.1210e-43,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  8.9683e-44,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00, -4.1467e-13,  4.5853e-41]]])

In [5]:
x.size()

torch.Size([2, 3, 4])

In [8]:
a = torch.rand(2,3,4)
b = torch.rand(2,3,4)
_ = torch.add(a,b,out = x)

In [9]:
_

tensor([[[0.6537, 0.8041, 0.4966, 1.5929],
         [1.7036, 1.0130, 1.2256, 0.7960],
         [0.8350, 0.6909, 1.0339, 1.0517]],

        [[1.1942, 0.9287, 0.3522, 1.0566],
         [0.6128, 1.0176, 0.4432, 1.5374],
         [0.8436, 0.8313, 0.6333, 0.5078]]])

In [10]:
x

tensor([[[0.6537, 0.8041, 0.4966, 1.5929],
         [1.7036, 1.0130, 1.2256, 0.7960],
         [0.8350, 0.6909, 1.0339, 1.0517]],

        [[1.1942, 0.9287, 0.3522, 1.0566],
         [0.6128, 1.0176, 0.4432, 1.5374],
         [0.8436, 0.8313, 0.6333, 0.5078]]])

In [11]:
a.add_(b)

tensor([[[0.6537, 0.8041, 0.4966, 1.5929],
         [1.7036, 1.0130, 1.2256, 0.7960],
         [0.8350, 0.6909, 1.0339, 1.0517]],

        [[1.1942, 0.9287, 0.3522, 1.0566],
         [0.6128, 1.0176, 0.4432, 1.5374],
         [0.8436, 0.8313, 0.6333, 0.5078]]])

In [12]:
from torch.autograd import Variable

In [13]:
x = torch.rand(5)
x = Variable(x,requires_grad=True)
y = x * 2 

In [14]:
grads = torch.FloatTensor([1,2,3,4,5])
y.backward(grads)
x.grad

tensor([ 2.,  4.,  6.,  8., 10.])

In [23]:
import torch.nn as nn
import torch.nn.functional as F

In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)       
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()

In [30]:
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [31]:
len(list(net.parameters()))

10

In [36]:
input = Variable(torch.rand(1,1,32,32))
out = net(input)
out

tensor([[-0.0717,  0.0400,  0.0973, -0.0552,  0.0905, -0.0407, -0.0644, -0.0650,
          0.0039, -0.0519]], grad_fn=<AddmmBackward>)

In [37]:
out.backward(torch.randn(1,10))

In [38]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate) # my optimizer

In [39]:
import torch.optim as optim

In [ ]:
optimizer = optim.SGD(net.parameters(),lr = 0.01)
optimizer.zero_grad()# zero the gradient buffers，如果不写这个函数，也是可以正常工作的，不知这个函数的必要性在哪？
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step()

# Overall CNN Structure

In [40]:
import torch.nn  as nn
import torch.nn.functional as F

In [44]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv1(x)),2) #if the size is a square you can only specify a number
        x = x.view(-1,self,num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()                   

In [45]:
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
optimizer = optim.SGD(net.parameters(),lr = 0.01)

for i in range(num_iteations):
    optimizer.zero_grad() #zero the gradient buffers，如果不归0的话，gradients会累加
    output = net(input)
    loss = criterion(output,target)
    loss.backward()
    optimizer.step()

# other

In [46]:
import numpy as np
import torch

In [47]:
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a,1,out=a)

array([2., 2., 2., 2., 2.])

In [49]:
a # 如果a 变的话， b也会跟着变，说明b只是保存了一个地址而已，并没有深拷贝

array([2., 2., 2., 2., 2.])

In [50]:
b # Variable只是保存Tensor的地址，如果Tensor变的话，Variable也会跟着变

tensor([2., 2., 2., 2., 2.], dtype=torch.float64)

In [51]:
a = np.ones(5)
b = torch.from_numpy(a)
a_ = b.numpy()

In [52]:
a,b,a_

(array([1., 1., 1., 1., 1.]),
 tensor([1., 1., 1., 1., 1.], dtype=torch.float64),
 array([1., 1., 1., 1., 1.]))

In [53]:
np.add(a,1,out=a)
a

array([2., 2., 2., 2., 2.])

In [54]:
if torch.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    x + y

In [55]:
torch.Tensor(1,2,3) # 生成一个 shape 为 [1,2,3] 的 tensor
torch.Tensor([1,2,3]) # 生成一个值为 [1,2,3] 的 tensor

tensor([1., 2., 3.])